# Farina in Natura - estratégia

![](Logo_farina_in_natura_edited.jpg)

## Importando os dados e observações iniciais

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import datetime
import nbformat
import chart_studio
import plotly.express as px
import cufflinks as cf

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode (connected = True )
cf.go_offline()
chart_studio.tools.set_credentials_file(username='claudio.oliveira', api_key='fXrLqCmkdJQyTheCzYMS')
from IPython.display import display


In [3]:
from fpdf import FPDF

In [4]:
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [5]:
# para resolver o problema: "(unicode error) 'utf-8' codec can't decode "
# https://www.youtube.com/watch?v=ZSC7X4N8RKo,
#Erro: 
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 900: invalid continuation byte
# enem = pd.read_csv("MICRODADOS_ENEM_2020.csv", encoding="utf-8", sep = ';')
# enem = pd.read_csv("MICRODADOS_ENEM_2020.csv", encoding="ISO-8859-1", sep = ';')

transactions = pd.read_csv(r'C:\Users\ctobr\OneDrive\DATA_Science\1_farina_in_natura\farina_in_natura00\CSV\farina.CSV', 
encoding="ISO-8859-1", sep = ';')

In [6]:
transactions.head()

,venda,data_hora,cliente,mesa,item,produto,quantidade,valor_unit,desconto,total_item
0,460.0,01/02/2022 08:13,CONSUMIDOR,M 1,119.0,SUCO DE LARANJA 400ML,1.0,"14,5",0.0,"14,5"
1,460.0,01/02/2022 08:13,CONSUMIDOR,M 1,121.0,PAO SOURDOUGH OVOS,1.0,"15,5",0.0,"15,5"
2,461.0,01/02/2022 08:14,CONSUMIDOR,M 2,119.0,SUCO DE LARANJA 400ML,1.0,"14,5",0.0,"14,5"
3,461.0,01/02/2022 08:14,CONSUMIDOR,M 2,121.0,PAO SOURDOUGH OVOS,1.0,"15,5",0.0,"15,5"
4,465.0,01/02/2022 09:01,CONSUMIDOR,M 1,46.0,QUICHE BACON 90G,2.0,"19,5",0.0,39


In [7]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25998 entries, 0 to 25997
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   venda       25992 non-null  float64
 1   data_hora   25992 non-null  object 
 2   cliente     25992 non-null  object 
 3   mesa        22779 non-null  object 
 4   item        25922 non-null  float64
 5   produto     25992 non-null  object 
 6   quantidade  25992 non-null  float64
 7   valor_unit  25992 non-null  object 
 8   desconto    25992 non-null  float64
 9   total_item  25992 non-null  object 
dtypes: float64(4), object(6)
memory usage: 2.0+ MB


# Limpeza e manipulação dos dados

In [8]:
### CHECK OUT IF THE NUMBER OF TOTAL ENTRIES IS EQUAL TO THE NUMBER OF ENTRIES OF THE VARIABLES  

transactions.isnull().sum()

venda            6
data_hora        6
cliente          6
mesa          3219
item            76
produto          6
quantidade       6
valor_unit       6
desconto         6
total_item       6
dtype: int64

In [9]:
## DROPPING THE ROWS THAT ALL COLUMN VALUES ARE NULL
#  https://www.youtube.com/watch?v=uDr67HBIPz8&t=818s

transactions = transactions.dropna(how='all')

In [10]:
# Delete the column "item"

del transactions["item"]

In [11]:
transactions.info()
transactions.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25992 entries, 0 to 25997
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   venda       25992 non-null  float64
 1   data_hora   25992 non-null  object 
 2   cliente     25992 non-null  object 
 3   mesa        22779 non-null  object 
 4   produto     25992 non-null  object 
 5   quantidade  25992 non-null  float64
 6   valor_unit  25992 non-null  object 
 7   desconto    25992 non-null  float64
 8   total_item  25992 non-null  object 
dtypes: float64(3), object(6)
memory usage: 2.0+ MB


,venda,data_hora,cliente,mesa,produto,quantidade,valor_unit,desconto,total_item
0,460.0,01/02/2022 08:13,CONSUMIDOR,M 1,SUCO DE LARANJA 400ML,1.0,"14,5",0.0,"14,5"
1,460.0,01/02/2022 08:13,CONSUMIDOR,M 1,PAO SOURDOUGH OVOS,1.0,"15,5",0.0,"15,5"
2,461.0,01/02/2022 08:14,CONSUMIDOR,M 2,SUCO DE LARANJA 400ML,1.0,"14,5",0.0,"14,5"
3,461.0,01/02/2022 08:14,CONSUMIDOR,M 2,PAO SOURDOUGH OVOS,1.0,"15,5",0.0,"15,5"
4,465.0,01/02/2022 09:01,CONSUMIDOR,M 1,QUICHE BACON 90G,2.0,"19,5",0.0,39


In [12]:
## Using isna() to select all rows with NaN in the column ('mesa') and the value "CONSUMIDOR" on the column "cliente" 
## and replace the "mesa" value with "M0"
## https://datatofish.com/rows-with-nan-pandas-dataframe/
## https://www.geeksforgeeks.org/how-to-replace-values-in-column-based-on-condition-in-pandas/

transactions.loc[(transactions["cliente"] == 'CONSUMIDOR') &  (transactions['mesa'].isna()), 'mesa']= 'M0'


In [13]:
# Checking if the changes were correct.

transactions.loc[transactions["mesa"] == 'M0']

,venda,data_hora,cliente,mesa,produto,quantidade,valor_unit,desconto,total_item
121,520.0,02/02/2022 16:57,CONSUMIDOR,M0,BOMBOLONI NUTELLA 80G,2.0,12,0.0,24
122,520.0,02/02/2022 16:57,CONSUMIDOR,M0,TARTELETE DE MORANGO 120G,1.0,18,0.0,18
123,520.0,02/02/2022 16:57,CONSUMIDOR,M0,CROISSANT MUSSATRELA DE,1.0,"25,5",0.0,"25,5"
894,856.0,11/02/2022 19:19,CONSUMIDOR,M0,CROISSANT BRIE PARMA MEL DE,2.0,"25,5",0.0,51
895,856.0,11/02/2022 19:19,CONSUMIDOR,M0,VH SANTA HORTENSIA SAUVIG,1.0,59,0.0,59


In [14]:
# Assigning a value "ND" to the empty records in column "mesa"
# https://www.w3schools.com/python/pandas/pandas_cleaning_empty_cells.asp

transactions['mesa'].fillna("ND",inplace = True )

In [15]:
# Convert the "venda" variable from float64 type to integer type
# https://stackoverflow.com/questions/43956335/convert-float64-column-to-int64-in-pandas

transactions['venda'] = np.int64(transactions['venda'])

In [16]:
# Convert the "quantidade" variable from float64 type to integer type
# https://stackoverflow.com/questions/43956335/convert-float64-column-to-int64-in-pandas

transactions['quantidade'] = np.int64(transactions['quantidade'])

In [17]:
# change the decimal delimiter of the the variable "valor_unit" from "," to "."
# https://cursos.alura.com.br/forum/topico-transformar-pontos-por-virgulas-dentro-de-um-dataframe-121090
transactions['valor_unit'] = transactions['valor_unit'].str.replace(",",".")

# Convert the "valor_uni" variable from object type to float64 type
# https://www.statology.org/pandas-convert-object-to-float/

transactions['valor_unit'] = transactions['valor_unit'].astype(float)

In [18]:
# change the decimal delimiter of the the variable "total_item" from "," to "."
# https://cursos.alura.com.br/forum/topico-transformar-pontos-por-virgulas-dentro-de-um-dataframe-121090
transactions['total_item'] = transactions['total_item'].str.replace(",",".")

# Convert the "total_item" variable from object type to float64 type
# https://www.statology.org/pandas-convert-object-to-float/

transactions['total_item'] = transactions['total_item'].astype(float)

In [19]:
# Convert "data_hora" to datetime object
transactions['data_hora'] = pd.to_datetime(transactions['data_hora'], format='%d/%m/%Y %H:%M')

#How to Solve Python ValueError: unconverted data remains
#read://https_researchdatapod.com/?url=https%3A%2F%2Fresearchdatapod.com%2Fhow-to-solve-python-valueerror-unconverted-data-remains%2F

In [20]:
# Creating a horario column that is a copy of data_hora;

# https://www.dataquest.io/blog/python-datetime-tutorial/


transactions['horario'] = (transactions['data_hora'])

#  add hora and minuto column; using class attributes .hour and .minute

transactions['hora'] = transactions.horario.dt.hour
transactions['minuto'] = transactions.horario.dt.minute

# drop horario column
transactions = transactions.drop(['horario'], axis=1)


### Reamaning the column "data_hora" to "data"

In [21]:
# https://www.statology.org/pandas-rename-columns/
# renaming the column

transactions.rename(columns = {'data_hora':'data'}, inplace = True)


### Extracting only the Date from 'Data' column

In [22]:

## https://datagy.io/pandas-extract-date-from-datetime/
transactions['data'] = transactions['data'].dt.normalize()

In [23]:
# Definindo a função time_of_day
def time_of_day(hora):
    """
    Determina se a  compra foi feita pela manhã, tarde ou noite.
    """
    
    if hora < 12:
        
        return 'manhã'
    
    elif hora < 17:
        
        return 'tarde'
    
    else:
        
        return 'noite'

In [24]:
# Apply time of day function to 'hora' column

transactions['time_of_day'] = transactions.hora.apply(time_of_day)


In [25]:
# https://pt.stackoverflow.com/questions/508484/mostrar-dias-da-semana#:~:text=Como%20weekday()%20retorna%20um,fim%20de%20semana%20ou%20n%C3%A3o.

import locale
# setar locale para português para que os dias da semana retornem em portugês no código abaixo.
locale.setlocale(locale.LC_ALL, 'pt_BR.utf8')

'pt_BR.utf8'

In [26]:
# Add a day of week variable and a second classifier of weekday or weekend

from datetime import date
import calendar

transactions['day_of_week'] = [calendar.day_name[i.weekday()] for i in transactions['data']]

transactions['day_type'] = ['FDS' if (i == 'Sunday') | (i == 'Saturday') else 'DDS' for i in 
                                                                            transactions['day_of_week']]


In [27]:
transactions.rename(columns = {'day_of_week':'dia_da_semana'}, inplace = True)

In [28]:
transactions.rename(columns = {'time_of_day':'hora_do_dia'}, inplace = True)

In [29]:
transactions.rename(columns = {'day_type':'tipo_de_dia'}, inplace = True)

In [30]:
# Add an item count variable equal to one for future groupby operations

transactions['item_count'] = 1


In [31]:
transactions.head()

,venda,data,cliente,mesa,produto,quantidade,valor_unit,desconto,total_item,hora,minuto,hora_do_dia,dia_da_semana,tipo_de_dia,item_count
0,460,2022-02-01,CONSUMIDOR,M 1,SUCO DE LARANJA 400ML,1,14.5,0.0,14.5,8,13,manhã,terça-feira,DDS,1
1,460,2022-02-01,CONSUMIDOR,M 1,PAO SOURDOUGH OVOS,1,15.5,0.0,15.5,8,13,manhã,terça-feira,DDS,1
2,461,2022-02-01,CONSUMIDOR,M 2,SUCO DE LARANJA 400ML,1,14.5,0.0,14.5,8,14,manhã,terça-feira,DDS,1
3,461,2022-02-01,CONSUMIDOR,M 2,PAO SOURDOUGH OVOS,1,15.5,0.0,15.5,8,14,manhã,terça-feira,DDS,1
4,465,2022-02-01,CONSUMIDOR,M 1,QUICHE BACON 90G,2,19.5,0.0,39.0,9,1,manhã,terça-feira,DDS,1


In [32]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25992 entries, 0 to 25997
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   venda          25992 non-null  int64         
 1   data           25992 non-null  datetime64[ns]
 2   cliente        25992 non-null  object        
 3   mesa           25992 non-null  object        
 4   produto        25992 non-null  object        
 5   quantidade     25992 non-null  int64         
 6   valor_unit     25992 non-null  float64       
 7   desconto       25992 non-null  float64       
 8   total_item     25992 non-null  float64       
 9   hora           25992 non-null  int64         
 10  minuto         25992 non-null  int64         
 11  hora_do_dia    25992 non-null  object        
 12  dia_da_semana  25992 non-null  object        
 13  tipo_de_dia    25992 non-null  object        
 14  item_count     25992 non-null  int64         
dtypes: datetime64[ns](1

# Verificando alguma divergência dos dados

In [33]:
# Verificando se o valor "total_item" é igual a: "quantidade" * "valor_unit"

transactions["total_item_verif"] = transactions['quantidade']* transactions['valor_unit']

In [34]:
# Continuando o item anterior
# https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/

transactions['confirm'] = np.where(transactions['total_item'] > transactions['total_item_verif'], True, False)


In [35]:
# continuando o item anterior
resumo = transactions.loc[transactions["confirm"] == True].sum()
print (resumo)

venda                                                        13730391
cliente             CONSUMIDORCONSUMIDORCONSUMIDORCONSUMIDORIFOODC...
mesa                M 5M 13M 11M 5NDM 19NDM 13M 13M 17NDM 23M 23M ...
produto             CROISSANTCROISSANTCROISSANTCROISSANTCROISSANT ...
quantidade                                                       2139
valor_unit                                                    21083.7
desconto                                                          0.0
total_item                                                    53878.3
hora                                                            29332
minuto                                                          59296
hora_do_dia         manhãmanhãtardemanhãmanhãmanhãmanhãnoitenoiten...
dia_da_semana       sexta-feirasexta-feirasexta-feirasábadosábados...
tipo_de_dia         DDSDDSDDSDDSDDSDDSDDSDDSDDSDDSDDSDDSDDSDDSDDSD...
item_count                                                       2032
total_item_verif    

C:\Users\ctobr\AppData\Local\Temp\ipykernel_6288\2200306820.py:2: FutureWarning:

The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



In [36]:
# Gerando um relatório em pdf dos dados divergentes
# How do I select specific rows and columns from a DataFrame?
# https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html#how-do-i-select-specific-rows-and-columns-from-a-dataframe
# https://www.geeksforgeeks.org/select-rows-columns-by-name-or-index-in-pandas-dataframe-using-loc-iloc/

ts2 = transactions.loc[transactions["confirm"] == True, ["venda", "produto", "quantidade", "valor_unit", "total_item", "total_item_verif"]]

In [37]:
# Exporting to a pdf file the transactions that has a inconsistent data.
from matplotlib.backends.backend_pdf import PdfPages


In [38]:
# https://levelup.gitconnected.com/how-to-write-a-pandas-dataframe-as-a-pdf-5cdf7d525488

def _draw_as_table(df, pagesize):
    alternating_colors = [['white'] * len(df.columns), ['lightgray'] * len(df.columns)] * len(df)
    alternating_colors = alternating_colors[:len(df)]
    fig, ax = plt.subplots(figsize=pagesize)
    ax.axis("tight")
    ax.axis('off')
    the_table = ax.table(cellText=df.values,
                        rowLabels=df.index,
                        colLabels=df.columns,
                        rowColours=['lightblue']*len(df),
                        colColours=['lightblue']*len(df.columns),
                        cellColours=alternating_colors,
                        loc='center')
    return fig


In [39]:
# https://www.alixaprodev.com/2022/05/convert-pandas-dataframe-to-pdf-in-python.html
#fig, ax =plt.subplots(figsize=(14,6))
#ax.axis('equal')
#ax.axis('off')
#the_table = ax.table(cellText=ts2.values,colLabels=ts2.columns,loc='center')




In [40]:
def dataframe_to_pdf(df, filename, numpages=(1, 1), pagesize=(11, 8.5)):
  with PdfPages(filename) as pdf:
    nh, nv = numpages
    rows_per_page = len(df) // nh
    cols_per_page = len(df.columns) // nv
    for i in range(0, nh):
        for j in range(0, nv):
            page = df.iloc[(i*rows_per_page):min((i+1)*rows_per_page, len(df)),
                           (j*cols_per_page):min((j+1)*cols_per_page, len(df.columns))]
            fig = _draw_as_table(page, pagesize)
            if nh > 1 or nv > 1:
                # Add a part/page number at bottom-center of page
                fig.text(0.5, 0.5/pagesize[0],
                         "Part-{}x{}: Page-{}".format(i+1, j+1, i*nv + j + 1),
                         ha='center', fontsize=8)
            pdf.savefig(fig, bbox_inches='tight')
            
            plt.close()

In [41]:
### Trying to Align the page number
# https://www.geeksforgeeks.org/matplotlib-figure-figure-text-in-python/
def dataframe_to_pdf(df, filename, numpages=(1, 1), pagesize=(11, 8.5)):
  with PdfPages(filename) as pdf:
    nh, nv = numpages
    rows_per_page = len(df) // nh
    cols_per_page = len(df.columns) // nv
    for i in range(0, nh):
        for j in range(0, nv):
            page = df.iloc[(i*rows_per_page):min((i+1)*rows_per_page, len(df)),
                           (j*cols_per_page):min((j+1)*cols_per_page, len(df.columns))]
            fig = _draw_as_table(page, pagesize)
            #if nh > 1 or nv > 1:
                # Add a part/page number at bottom-center of page
                #fig.text(0.5, 0.5/pagesize[0],
                       # "Part-{}x{}: Page-{}".format(i+1, j+1, i*nv + j + 1),
                       # verticalalignment ='bottom', 
                       # horizontalalignment ='right',
                       #transform = ax.transAxes,
                        #color ='green',
                        #fontsize = 5
                         #)
            pdf.savefig(fig, bbox_inches='tight')
            
            plt.close()

### Creating a pdf file from the divergent data

In [42]:
# dataframe_to_pdf(ts2, "diverge_01.pdf", numpages=(12, 1), pagesize=(11.7, 8.3))

In [43]:
## Create a HTML file of the resumé "ts2" dataframe.

f = open('exp.html','w')
a = ts2.to_html()
f.write(a)
f.close()

In [44]:
# drop the 'confirm" and "total_item_verif" column
transactions = transactions.drop(['confirm'], axis=1)
transactions = transactions.drop(['total_item_verif'], axis=1)


In [45]:
transactions.head()

,venda,data,cliente,mesa,produto,quantidade,valor_unit,desconto,total_item,hora,minuto,hora_do_dia,dia_da_semana,tipo_de_dia,item_count
0,460,2022-02-01,CONSUMIDOR,M 1,SUCO DE LARANJA 400ML,1,14.5,0.0,14.5,8,13,manhã,terça-feira,DDS,1
1,460,2022-02-01,CONSUMIDOR,M 1,PAO SOURDOUGH OVOS,1,15.5,0.0,15.5,8,13,manhã,terça-feira,DDS,1
2,461,2022-02-01,CONSUMIDOR,M 2,SUCO DE LARANJA 400ML,1,14.5,0.0,14.5,8,14,manhã,terça-feira,DDS,1
3,461,2022-02-01,CONSUMIDOR,M 2,PAO SOURDOUGH OVOS,1,15.5,0.0,15.5,8,14,manhã,terça-feira,DDS,1
4,465,2022-02-01,CONSUMIDOR,M 1,QUICHE BACON 90G,2,19.5,0.0,39.0,9,1,manhã,terça-feira,DDS,1


In [46]:
transactions.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 25992 entries, 0 to 25997
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   venda          25992 non-null  int64         
 1   data           25992 non-null  datetime64[ns]
 2   cliente        25992 non-null  object        
 3   mesa           25992 non-null  object        
 4   produto        25992 non-null  object        
 5   quantidade     25992 non-null  int64         
 6   valor_unit     25992 non-null  float64       
 7   desconto       25992 non-null  float64       
 8   total_item     25992 non-null  float64       
 9   hora           25992 non-null  int64         
 10  minuto         25992 non-null  int64         
 11  hora_do_dia    25992 non-null  object        
 12  dia_da_semana  25992 non-null  object        
 13  tipo_de_dia    25992 non-null  object        
 14  item_count     25992 non-null  int64         
dtypes: datetime64[ns](1

> # **Exploração dos Dados**

In [47]:
# Find Unique Values of each item sold in the Column "produto"
# https://www.statology.org/pandas-unique-values-in-column/
transactions.produto.value_counts()

CROISSANT                         2393
PASTEL DE NATA 60G                1339
CROISSANT  AMENDOAS 100G           742
FOLHADO MACA 80G                   719
SANDUICHE DE PEPERONI BRIE         596
COOKIES 80G                        559
CROISSANT IFOOD                    534
SOURDOUGH AZEITONA E ALECRIM       517
FOLHADO FRANGO 160G                500
BOMBOLONI DOCE DE LEITE 80G        487
FOCACCIA FORMAGGIO 246G            460
AGUA MINERAL S/GAS 500ML           446
TORRADA FARINA 165G                435
CAFE EQUILIBRADO CAPSULA P         431
SOURDOUGH 369G                     431
LARANJA 400 ML                     430
MINI CROISSANT 25G KIT C 5         427
CROISSANT BRIE PARMA MEL DE        423
PIZZA AL TAGLIO GORGONZOLA E       421
MINI PAIN AU CHOCOLAT 35G          419
BOMBOLONI NUTELLA 80G              409
CROISSANT  TRADICIONAL 80 G        396
QUICHE DE FRANGO ALHO PORO         360
AGUA MINERAL SEM GAS 500 ML        341
COCA COLA LATA 350 ML              337
SUCO DE LARANJA 400ML    

In [48]:
# find the percentage each item is selling
transactions.produto.value_counts(True)

CROISSANT                         0.092067
PASTEL DE NATA 60G                0.051516
CROISSANT  AMENDOAS 100G          0.028547
FOLHADO MACA 80G                  0.027662
SANDUICHE DE PEPERONI BRIE        0.022930
COOKIES 80G                       0.021507
CROISSANT IFOOD                   0.020545
SOURDOUGH AZEITONA E ALECRIM      0.019891
FOLHADO FRANGO 160G               0.019237
BOMBOLONI DOCE DE LEITE 80G       0.018737
FOCACCIA FORMAGGIO 246G           0.017698
AGUA MINERAL S/GAS 500ML          0.017159
TORRADA FARINA 165G               0.016736
CAFE EQUILIBRADO CAPSULA P        0.016582
SOURDOUGH 369G                    0.016582
LARANJA 400 ML                    0.016544
MINI CROISSANT 25G KIT C 5        0.016428
CROISSANT BRIE PARMA MEL DE       0.016274
PIZZA AL TAGLIO GORGONZOLA E      0.016197
MINI PAIN AU CHOCOLAT 35G         0.016120
BOMBOLONI NUTELLA 80G             0.015736
CROISSANT  TRADICIONAL 80 G       0.015235
QUICHE DE FRANGO ALHO PORO        0.013850
AGUA MINERA

In [49]:
## Get unique names in the "produto" column using Series.unique() 
## https://sparkbyexamples.com/pandas/pandas-count-unique-values-in-column/
num_itens_vend = transactions.produto.unique().size

print('O número de diferentes itens vendido pela loja é:', num_itens_vend)

O número de diferentes itens vendido pela loja é: 212


In [50]:
# Look at the average number of items purchased...

avg_items = transactions.groupby(by='venda')['produto'].count().mean()

print('A média de itens por venda é: %.2f itens' %avg_items)

A média de itens por venda é: 2.79 itens


In [51]:
# ...and how that varies by time of day

single_transaction = pd.DataFrame(transactions.groupby(by='venda')['item_count'].sum())

single_transaction = single_transaction.merge(transactions[['hora','minuto','hora_do_dia','data', 'venda']]\
                                              .drop_duplicates('venda'), left_on=single_transaction.index, 
                                              right_on='venda', how='left', copy=False)

print('O número médio de itens vendidos por hora do dia é: \n{0}'.format(
    single_transaction.groupby(by='hora_do_dia')['item_count'].mean()))

O número médio de itens vendidos por hora do dia é: 
hora_do_dia
manhã    3.046419
noite    2.663510
tarde    2.783376
Name: item_count, dtype: float64


In [52]:
# Find the frequency of each item purchased and by time of day

item_frequencies_all = transactions.produto.value_counts()
item_frequencies_morning = transactions[transactions.hora_do_dia == 'manhã'].produto.value_counts()
item_frequencies_afternoon = transactions[transactions.hora_do_dia == 'tarde'].produto.value_counts()
item_frequencies_evening = transactions[transactions.hora_do_dia == 'noite'].produto.value_counts()

In [53]:
# Plot the frequency of purchase for each item

# https://community.plotly.com/t/errors-accessing-plotly-api/33271
# ok, so I found a solution to my problem - when I let my program run through, the error message produced is:
#chart_studio.exceptions.PlotlyRequestError: Authentication credentials were not provided.
#I Googled this error and there were suggestions to run plotly in offline mode by replacing:
#import chart_studio.plotly as py
#with

#import plotly.offline as py
#then use py.plot instead of py.iplot

# Solving the problem of the 'plotly' online library with chart_studio
# https://plotly.com/python/getting-started-with-chart-studio/
import plotly.graph_objs as go
import chart_studio.plotly as py
# import plotly.express as px

button_layer_1_height = 1.12
button_layer_2_height = 1.065

item_threshold_all = 100
item_threshold_other = 50

trace_all = go.Bar(
            x=item_frequencies_all[item_frequencies_all > item_threshold_all].index,
            y=item_frequencies_all[item_frequencies_all > item_threshold_all].values,
            visible=True,
            name='Todo dia'
    )

trace_morning = go.Bar(
            x=item_frequencies_morning[item_frequencies_morning > item_threshold_other].index,
            y=item_frequencies_morning[item_frequencies_morning > item_threshold_other].values,
            visible=False,
            name='Manhã'
    )

trace_afternoon = go.Bar(
            x=item_frequencies_afternoon[item_frequencies_afternoon > item_threshold_other].index,
            y=item_frequencies_afternoon[item_frequencies_afternoon > item_threshold_other].values,
            visible=False,
            name='Tarde'
    )

trace_evening = go.Bar(
            x=item_frequencies_evening[item_frequencies_evening > 10].index,
            y=item_frequencies_evening[item_frequencies_evening > 10].values,
            visible=False,
            name='Noite'
    )

data = [trace_all, trace_morning, trace_afternoon, trace_evening]

updatemenus = list([
    
    dict(
         buttons=list([   
            dict(label = 'Todo dia',
                 method = 'update',
                 args = [{'visible': [True, False, False, False]},
                         {'title': 'Todo Dia'}]),
             
            dict(label = 'Manhã',
                 method = 'update',
                 args = [{'visible': [False, True, False, False]},
                         {'title': 'Manhã'}]),
             
            dict(label = 'Tarde',
                 method = 'update',
                 args = [{'visible': [False, False, True, False]},
                         {'title': 'Tarde'}]),
             
            dict(label = 'Noite',
                 method = 'update',
                 args = [{'visible': [False, False, False, True]},
                         {'title': 'Noite'}])
        ]),
        type='buttons',
        direction = 'right',
        pad = {'r': 10, 't': 10},
        showactive = True,
        x = 0.3,
        xanchor = 'left',
        y = button_layer_1_height,
        yanchor = 'top' )
])
                
layout = dict(title='Vendas Farina por hora do dia', showlegend=False,
              updatemenus=updatemenus)



fig = dict(data=data, layout=layout)
py.iplot(fig, filename='frequencies-bar.html')

In [144]:
# Sales trend by date
# https://www.youtube.com/watch?v=Xjm7fOw6gjY
# use reset index for transforming the data in a variable not a index

grouped_by_date = pd.DataFrame(transactions.groupby(by='data')['item_count'].sum().reset_index())

In [136]:
grouped_by_mes =(transactions.groupby(transactions.data.dt.month))['item_count'].sum().reset_index()
grouped_by_mes

,data,item_count
0,2,2419
1,3,2680
2,4,2054
3,5,3752
4,6,4210
5,7,1527
6,8,3206
7,9,3019
8,10,3125


In [162]:
px.scatter(grouped_by_mes, x = 'data', y = 'item_count', animation_frame= 'data', range_x=[2,10],
 range_y= [1000, 5000], log_y=False, size = 'item_count', labels= {'item_count' : 'Quantidade de itens vendidos','data': 'Mês'})

In [85]:
grouped_by_mes.head()

,data,item_count
0,2022-02-01,74
1,2022-02-02,88
2,2022-02-03,81
3,2022-02-04,110
4,2022-02-05,112


In [95]:
from chart_studio.grid_objs import Grid, Column


import plotly.figure_factory as ff

# Create a plot.ly-friendly table for Sales per day

table = ff.create_table(grouped_by_date)
py.iplot(table, filename='Vendas_por_dia_farina')

f = open('Vendas_por_dia_farina.html','w')
a = grouped_by_date.to_html()
f.write(a)
f.close()

In [166]:
px.line(grouped_by_date, x = 'data', y = 'item_count', title = 'Itens vendidos por dia', labels= {'item_count' : 'Quantidade de itens vendidos','data': 'Data'})

In [113]:
# Vendas atípicas no dia 23 de Junho de 2022 entre 16:00 e 17:00

df_invest = pd.DataFrame(transactions).query("data == 'Jun 23, 2022'").groupby(by='hora')['item_count'].sum().reset_index()
df_invest


,hora,item_count
0,7,8
1,9,5
2,10,5
3,11,7
4,12,4
5,13,8
6,14,5
7,15,3
8,16,2254
9,17,3


In [172]:
dataframe_to_pdf(df_invest, "investiga.pdf", numpages=(1, 1), pagesize=(11.7, 8.3))

In [171]:
graf = px.line(df_invest, x = 'hora', y = 'item_count', labels= {'item_count':'Itens Vendidos'}, title= 'Itens vendidos no dia 23 de Junho 2022')
graf

f = open('investiga.html','w')
a = graf.to_html()
f.write(a)
f.close()

In [ ]:
# Formatting for the plot.ly chart based on code adapted from an example in their gallery

number_of_items_sold = list(grouped_by_date['item_count'])

my_columns = []

for k in range(len(grouped_by_date.index) - 1):
    
    my_columns.append(Column(grouped_by_date.index[:k + 1], 'x{}'.format(k + 1)))   
    my_columns.append(Column(number_of_items_sold[:k + 1], 'y{}'.format(k + 1)))
    
grid = Grid(my_columns)
py.iplot(grid, 'itens_vendidos', auto_open=False)
py.grid_ops.upload(grid, 'itens_vendidos', auto_open=False)

In [ ]:
data=[dict(type='scatter',
           xsrc=grid.get_column_reference('x1'),
           ysrc= grid.get_column_reference('y1'),
           name='Bakery',
           mode='lines',
           line=dict(color= 'rgb(0, 153, 255)'),
           fill='tozeroy',
           fillcolor='rgba(153, 204, 255, 0.5)')]

axis=dict(ticklen=4,
          mirror=True,
          zeroline=False,
          showline=True,
          autorange=False,
          showgrid=False)

layout = dict(title='Sales per Day',
              font=dict(family='Balto'),
              showlegend=False,
              autosize=False,
              width=800,
              height=400,
              xaxis=dict(axis, **{'nticks':12, 'tickangle':-45,
                                  'range': [min(grouped_by_date.index),
                                            max(grouped_by_date.index)]}),
              yaxis=dict(axis, **{'title': 'Items Sold', 'range':[0,max(grouped_by_date.item_count)+5]}),
              updatemenus=[dict(type='buttons',
                                showactive=False,
                                y=1,
                                x=1.1,
                                xanchor='right',
                                yanchor='top',
                                pad=dict(t=0, r=10),
                                buttons=[dict(label='Play',
                                              method='animate',
                                              args=[None, dict(frame=dict(duration=50, redraw=False), 
                                                               transition=dict(duration=0),
                                                               fromcurrent=True,
                                                               mode='immediate')])])])

frames=[{'data':[{'xsrc': grid.get_column_reference('x{}'.format(k + 1)),
                  'ysrc': grid.get_column_reference('y{}'.format(k + 1))}],
         'traces': [0]
        } for k in range(len(grouped_by_date.index) - 1)]

fig=dict(data=data, layout=layout, frames=frames)
px.icreate_animations(fig, 'Vendas-por-dia_Farina3')